In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
# (수정) StandardScaler 대신 RobustScaler 임포트
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
# (수정) class_weight 계산을 위해 임포트
from sklearn.utils.class_weight import compute_class_weight
import joblib

print(f"TensorFlow 버전: {tf.__version__}")

# 1. 파일 목록 정의 (3가지 클래스)
# 라벨 0: 걷기
WALK_FILES = [
    'nomal_wark3_20251106_201207.csv',
    'normal_wark_20251106_195044.csv'
]
# 라벨 1: 뛰기
RUN_FILES = [
    'nomal_run_20251106_195716.csv',
    'nomal_run2_20251106_200243.csv',
    'nomal_run3_20251106_201900.csv',
    'nomal_run6_20251106_203042.csv'
]
# 라벨 2: 정지
STOP_FILES = [
    'stop_20251106_213440.csv',
    'stop2_20251106_213649.csv',
    'stop3_20251106_215912.csv',
    'stop4_20251106_220218.csv'

]
# 테스트용 '이상한 걸음'
ABNORMAL_FILES = [
    'str_walk_20251106_200527.csv'
    'str_wark2_20251106_200903.csv' # 이 파일도 있다면 목록에 추가
]

# 2. 헬퍼 함수
def load_data(files):
    """파일 목록을 받아 하나의 DataFrame으로 합칩니다."""
    df_list = []
    for f in files:
        try:
            df = pd.read_csv(f)
            if not df.empty:
                df_list.append(df)
        except Exception as e:
            print(f"파일 로드 오류 {f}: {e}")
    if not df_list:
        return pd.DataFrame()
    return pd.concat(df_list, axis=0, ignore_index=True)

TIME_STEPS = 100
def create_sequences(data, label, time_steps=TIME_STEPS):
    """스케일링된 데이터와 라벨을 받아 시퀀스를 생성합니다."""
    sequences = []
    labels = []
    for i in range(len(data) - time_steps + 1):
        sequences.append(data[i:(i + time_steps)])
        labels.append(label)
    return np.array(sequences), np.array(labels)

# 3. 데이터 로드 및 전처리
print("\n--- 3-클래스 데이터 로드 및 전처리 ---")

df_walk = load_data(WALK_FILES)
df_run = load_data(RUN_FILES)
df_stop = load_data(STOP_FILES)

print(f"로드된 '걷기' (라벨 0) 데이터: {len(df_walk)} 행")
print(f"로드된 '뛰기' (라벨 1) 데이터: {len(df_run)} 행")
print(f"로드된 '정지' (라벨 2) 데이터: {len(df_stop)} 행")

# (중요) 스케일러 학습: 3가지 클래스 데이터를 모두 합쳐서 fit
df_all_raw = pd.concat([df_walk, df_run, df_stop], ignore_index=True)

if df_all_raw.empty:
    print("오류: 학습 데이터가 없습니다. 파일 업로드를 확인하세요.")
else:
    # *** FIX 1: StandardScaler를 RobustScaler로 변경 (이상치 대응) ***
    scaler = RobustScaler()
    scaler.fit(df_all_raw)
    print("스케일러가 '걷기+뛰기+정지' 전체 데이터로 학습되었습니다. (RobustScaler 사용)")

    # 스케일러 적용
    scaled_walk = scaler.transform(df_walk)
    scaled_run = scaler.transform(df_run)
    scaled_stop = scaler.transform(df_stop)

    # 라벨을 붙여 시퀀스 생성
    X_walk, y_walk = create_sequences(scaled_walk, 0) # 0 = Walk
    X_run, y_run = create_sequences(scaled_run, 1)   # 1 = Run
    X_stop, y_stop = create_sequences(scaled_stop, 2)   # 2 = Stop

    print(f"'걷기' 시퀀스 생성: {X_walk.shape}")
    print(f"'뛰기' 시퀀스 생성: {X_run.shape}")
    print(f"'정지' 시퀀스 생성: {X_stop.shape}")

    # 4. 전체 데이터셋 결합 및 분리
    X_full = np.concatenate((X_walk, X_run, X_stop))
    y_full = np.concatenate((y_walk, y_run, y_stop))

    # (중요) shuffle=True로 3가지 데이터를 골고루 섞음
    X_train, X_test, y_train, y_test = train_test_split(
        X_full, y_full, test_size=0.2, shuffle=True, random_state=42
    )

    print(f"\n총 학습 데이터: {X_train.shape}, 총 테스트 데이터: {X_test.shape}")
    print(f"라벨 분포 (학습): {np.unique(y_train, return_counts=True)}")
    print(f"라벨 분포 (테스트): {np.unique(y_test, return_counts=True)}")

    # 5. 다중 클래스 LSTM 모델 구축
    n_features = X_train.shape[2]
    N_CLASSES = 3 # (0: 걷기, 1: 뛰기, 2: 정지)

    model = keras.Sequential([
        keras.layers.LSTM(64, activation='relu', input_shape=(TIME_STEPS, n_features), return_sequences=True),
        keras.layers.Dropout(0.3),
        keras.layers.LSTM(32, activation='relu', return_sequences=False),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(N_CLASSES, activation='softmax')
    ])

    # *** FIX 4: 학습률(Learning Rate)을 0.0001로 낮추고, clipnorm=1.0 유지 ***
    optimizer = keras.optimizers.Adam(learning_rate=0.0001, clipnorm=1.0)

    model.compile(optimizer=optimizer, # 수정된 옵티마이저 적용
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()

    # *** FIX 3 (Part 1): 클래스 가중치 계산 (데이터 불균형 해소) ***
    # y_train (학습 데이터 라벨)을 기반으로 클래스별 가중치 계산
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    # 딕셔너리 형태로 변환 {0: 0.x, 1: 1.y, 2: 3.z}
    class_weights_dict = dict(enumerate(class_weights))
    print(f"\n데이터 불균형 해소를 위한 클래스 가중치 적용:")
    print(f"  (0:걷기, 1:뛰기, 2:정지) ➡️ {class_weights_dict}")

    # 6. 모델 학습
    print("\n--- 3-클래스 모델 학습 시작 (수정 적용) ---")
    history = model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode='max')],
        # *** FIX 3 (Part 2): class_weight 옵션 추가 ***
        class_weight=class_weights_dict
    )

    # 7. 모델 평가
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"\n--- 테스트 데이터 평가 결과 ---")
    print(f"  - 손실(Loss): {loss:.4f}")
    print(f"  - 정확도(Accuracy): {accuracy * 100:.2f}%")

    print("\n(라벨 0: 걷기, 라벨 1: 뛰기, 라벨 2: 정지)")
    # '걷기' 샘플(라벨 0) 예측
    X_test_walk_sample = X_test[y_test == 0][:1]
    if len(X_test_walk_sample) > 0:
        pred = model.predict(X_test_walk_sample)
        print(f"\n'걷기' 샘플 예측 (실제: 0) ➡️ [W, R, S] 확률: {pred[0]} ➡️ 예측: {np.argmax(pred[0])}")

    # '뛰기' 샘플(라벨 1) 예측
    X_test_run_sample = X_test[y_test == 1][:1]
    if len(X_test_run_sample) > 0:
        pred = model.predict(X_test_run_sample)
        print(f"\n'뛰기' 샘플 예측 (실제: 1) ➡️ [W, R, S] 확률: {pred[0]} ➡️ 예측: {np.argmax(pred[0])}")

    # '정지' 샘플(라벨 2) 예측
    X_test_stop_sample = X_test[y_test == 2][:1]
    if len(X_test_stop_sample) > 0:
        pred = model.predict(X_test_stop_sample)
        print(f"\n'정지' 샘플 예측 (실제: 2) ➡️ [W, R, S] 확률: {pred[0]} ➡️ 예측: {np.argmax(pred[0])}")

    # '이상한 걸음'은 어떻게 예측하는지 확인
    df_abnormal = load_data(ABNORMAL_FILES)
    if not df_abnormal.empty:
        scaled_abnormal = scaler.transform(df_abnormal)
        X_abnormal, _ = create_sequences(scaled_abnormal, -1)
        if len(X_abnormal) > 0:
            pred_abnormal = model.predict(X_abnormal[:5])
            print(f"\n--- '이상한 걸음(Abnormal)' 데이터 예측 테스트 ---")
            for i, pred in enumerate(pred_abnormal):
                print(f"  샘플 {i} ➡️ [W, R, S]: {pred} ➡️ 예측: {np.argmax(pred)}")

    # 8. 최종 파일 Export
    print("\n--- 라즈베리파이용 파일 Export 시작 ---")

    # (1) Scaler 저장 (새로운 RobustScaler)
    scaler_filename = 'multi_class_scaler.joblib'
    joblib.dump(scaler, scaler_filename)
    print(f"스케일러 저장 완료: {scaler_filename}")

    # (2) TFLite 모델 저장 (새로운 모델)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)

    # TFLite 호환성 설정 (이전 성공 옵션)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter._experimental_lower_tensor_list_ops = False

    tflite_model = converter.convert()

    tflite_filename = 'multi_class_model.tflite'
    with open(tflite_filename, 'wb') as f:
        f.write(tflite_model)
    print(f"TFLite 모델 저장 완료: {tflite_filename}")

    print("\n--- 모든 작업 완료 ---")
    print("Colab의 '파일' 탭에서 2개의 파일을 다운로드하세요:")
    print("1. multi_class_scaler.joblib")
    print("2. multi_class_model.tflite")

TensorFlow 버전: 2.19.0

--- 3-클래스 데이터 로드 및 전처리 ---
로드된 '걷기' (라벨 0) 데이터: 3249 행
로드된 '뛰기' (라벨 1) 데이터: 1965 행
로드된 '정지' (라벨 2) 데이터: 2220 행
스케일러가 '걷기+뛰기+정지' 전체 데이터로 학습되었습니다. (RobustScaler 사용)
'걷기' 시퀀스 생성: (3150, 100, 12)
'뛰기' 시퀀스 생성: (1866, 100, 12)
'정지' 시퀀스 생성: (2121, 100, 12)

총 학습 데이터: (5709, 100, 12), 총 테스트 데이터: (1428, 100, 12)
라벨 분포 (학습): (array([0, 1, 2]), array([2504, 1493, 1712]))
라벨 분포 (테스트): (array([0, 1, 2]), array([646, 373, 409]))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                  │ (None, 100, 64)        │        19,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_19 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,707 (127.76 KB)

 Trainable params: 32,707 (127.76 KB)

 Non-trainable params: 0 (0.00 B)


데이터 불균형 해소를 위한 클래스 가중치 적용:
  (0:걷기, 1:뛰기, 2:정지) ➡️ {0: np.float64(0.7599840255591054), 1: np.float64(1.2746148693904888), 2: np.float64(1.1115654205607477)}

--- 3-클래스 모델 학습 시작 (수정 적용) ---
Epoch 1/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 14s 47ms/step - accuracy: 0.4859 - loss: 5.2516 - val_accuracy: 0.9405 - val_loss: 1534.3326
Epoch 2/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8815 - loss: 928.1519 - val_accuracy: 0.9503 - val_loss: 22.7954
Epoch 3/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9192 - loss: 2667.6531 - val_accuracy: 0.9727 - val_loss: 13.9313
Epoch 4/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.7713 - loss: 86926.3594 - val_accuracy: 0.7745 - val_loss: 128007.2812
Epoch 5/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.6033 - loss: 508566.5625 - val_accuracy: 0.6057 - val_loss: 6222.2998
Epoch 6/30
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.6066 - loss: 12091.4932 - val_accuracy: 0.6800 - val_loss: 763.3568
Ep

In [7]:
# (TIME_STEPS는 학습 때와 동일하게 설정되어 있어야 함. 예: TIME_STEPS = 100)

# 1. 예측을 위한 시퀀스 생성 함수 (학습 때와 동일)
# (이 함수가 이미 Colab 노트북 어딘가에 정의되어 있다면 생략 가능)
def create_sequences_for_inference(data, time_steps):
    sequences = []
    # 데이터가 time_steps보다 짧으면 예측 불가
    if len(data) >= time_steps:
        for i in range(len(data) - time_steps + 1):
            sequences.append(data[i:(i + time_steps)])
    return np.array(sequences)

# 2. 최종 '정상도' 예측 함수
def predict_running_accuracy(file_path, model_to_use, scaler_to_use, time_steps=100):
    """
    새로운 CSV 파일 하나를 받아 '정상도'의 평균 점수를 반환합니다.

    :param file_path: 새 데이터 파일의 경로 (예: 'my_new_run.csv')
    :param model_to_use: 이전에 학습된 keras 모델 객체
    :param scaler_to_use: 이전에 학습된 StandardScaler 객체
    :param time_steps: 학습 때 사용한 TIME_STEPS (예: 100)
    :return: '정상도' 평균 점수 (0~100 사이)
    """
    try:
        # 1. 새 데이터 로드
        df_new = pd.read_csv(file_path)

        # 2. 데이터 스케일링 (중요: 학습된 scaler로 'transform'만 수행)
        scaled_new_data = scaler_to_use.transform(df_new)

        # 3. 시계열 시퀀스 생성
        new_sequences = create_sequences_for_inference(scaled_new_data, time_steps)

        if len(new_sequences) == 0:
            print(f"  [경고] '{file_path}' 데이터가 너무 짧아 (총 {len(df_new)} 행) 예측할 수 없습니다. (최소 {time_steps} 행 필요)")
            return 0.0 # 또는 None

        # 4. 모델 예측 (모델은 '비정상(1)'일 확률을 반환)
        # shape: (n_sequences, 1)
        predictions_prob = model_to_use.predict(new_sequences)

        # 5. '정상도'로 변환
        # (1 - 비정상 확률) * 100 = 정상도(%)
        normality_scores = (1 - predictions_prob.flatten()) * 100

        # 6. 전체 시퀀스의 '정상도' 평균 반환
        average_score = np.mean(normality_scores)

        return average_score

    except FileNotFoundError:
        print(f"  [오류] 파일을 찾을 수 없습니다: {file_path}")
        return 0.0
    except Exception as e:
        print(f"  [오류] 예측 중 문제가 발생했습니다: {e}")
        return 0.0

In [8]:
# --- 실제 예측 수행 ---

# 1. (가정) Colab에 새로운 파일을 업로드합니다.
#    예시로, 테스트에 사용했던 'str_walk_20251106_200527.csv' 파일을
#    'my_test_abnormal.csv' 라는 이름으로 업로드했다고 가정해봅시다.
#    (실제로는 'my_test_abnormal.csv' 파일을 Colab에 업로드해야 함)

# 2. (가정) 정상 파일도 업로드
#    'nomal_run2_20251106_200243.csv' 파일을
#    'my_test_normal.csv' 라는 이름으로 업로드했다고 가정

# (model과 scaler는 이전 코드 셀에서 학습 완료된 객체라고 가정합니다)

# 테스트 1: '정상' 파일 예측
file_to_test_1 = 'nomal_run2_20251106_200243.csv' # Colab에 업로드된 파일
final_score_1 = predict_running_accuracy(file_to_test_1, model, scaler, TIME_STEPS)
print(f"\n--- 예측 결과 1 ---")
print(f"파일 '{file_to_test_1}'의 최종 '정상도': {final_score_1:.2f}%")


# 테스트 2: '비정상' 파일 예측
file_to_test_2 = 'str_walk_20251106_200527.csv' # Colab에 업로드된 파일
final_score_2 = predict_running_accuracy(file_to_test_2, model, scaler, TIME_STEPS)
print(f"\n--- 예측 결과 2 ---")
print(f"파일 '{file_to_test_2}'의 최종 '정상도': {final_score_2:.2f}%")

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step

--- 예측 결과 1 ---
파일 'nomal_run2_20251106_200243.csv'의 최종 '정상도': 100.00%
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

--- 예측 결과 2 ---
파일 'str_walk_20251106_200527.csv'의 최종 '정상도': 0.02%


In [13]:
!pip install joblib

import joblib
from google.colab import files
import tensorflow as tf # tf가 import되어 있는지 확인

# (model 과 scaler 객체는 이미 학습/생성되었다고 가정)

# --- 1. Scaler 저장 ---
scaler_filename = 'running_scaler.joblib'
joblib.dump(scaler, scaler_filename)
print(f"Scaler 저장 완료: {scaler_filename}")

# --- 2. TFLite 모델 변환 및 저장 ---
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# (중요) 성능 최적화
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# --- 💡 (오류 수정) TensorList ops 문제 해결 ---
# TFLite가 기본적으로 처리 못하는 TF 연산(op)을 포함하도록 허용
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS, # TFLite 기본 연산
    tf.lite.OpsSet.SELECT_TF_OPS    # TensorFlow 연산 선택적 포함
]
# TensorList 관련 변환 옵션 비활성화 (오류 메시지 가이드)
converter._experimental_lower_tensor_list_ops = False
# --- (수정 완료) ---

print("TFLite 변환 시작 (Select TF ops 포함)...")
tflite_model = converter.convert()
print("TFLite 변환 완료.")

# 변환된 .tflite 모델을 파일로 저장
tflite_filename = 'running_model.tflite'
with open(tflite_filename, 'wb') as f:
    f.write(tflite_model)
print(f"TensorFlow Lite 모델 저장 완료: {tflite_filename}")

# --- 3. Colab에서 로컬 PC로 파일 다운로드 ---
print("\n파일 2개를 로컬 PC로 다운로드합니다...")
files.download(scaler_filename)
files.download(tflite_filename)

Scaler 저장 완료: running_scaler.joblib
TFLite 변환 시작 (Select TF ops 포함)...
Saved artifact at '/tmp/tmpj8ueu4xw'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100, 12), dtype=tf.float32, name='keras_tensor_21')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  132340035609680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132342706208208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132342706208400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132342706205520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132342706203216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132340269758800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132340269746704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132340269744208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132340269750736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13234026974574

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>